In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import HTMLParser as html
from sklearn.feature_extraction.text import TfidfVectorizer
import urllib
#import nltk
from textblob import TextBlob, Word

In [2]:
#retrieves story headlines from NYT website
response = requests.get("http://www.nytimes.com/")
parser = BeautifulSoup(response.text, "html.parser")

top_story = []

top_link = []

top_stories = parser.select('div.a-column')[0]
top_headings = top_stories.select('h2.story-heading')

for story in top_headings:
    try:
        top_story.append(story.get_text())
        pattern = re.compile("http.+\.html")
        head = re.search(pattern, str(story)).group(0)
        head.replace('"','')
        top_link.append(head)
    except:
        top_link.append(None)
    
top_df = pd.DataFrame({'headline':top_story,'link':top_link})
top_df = top_df.dropna()

In [3]:
#story input is an index 0 to n; where n is the number of headlines in top_df
def NYT(story):
    
    #retrieve story link and html from that link
    story_link = top_df.link[story]

    response = requests.get(story_link)
    parser = BeautifulSoup(response.text, "html.parser")
    
    #parse article to grab text and title
            
    title = top_df.headline[story]
    
    article = []

    for para in parser.select('p.story-body-text'):
        article.append(para.get_text().replace(u'.\u201d', u'\u201d.'))
 
    #create article string
    s = ' '
    article1 = s.join(article).strip()
    
    #TFIDF for full text article
    stopwords = ['mr','chili','pic','sale','amazon','co','wire','a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'do', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed', 'interest', 'into', 'is', 'it', 'its', 'itself', 'keep', 'last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made', 'many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'over', 'own', 'part', 'per', 'perhaps', 'please', 'put', 'rather', 're', 'same', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'thick', 'thin', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'un', 'under', 'until', 'up', 'upon', 'us', 'very', 'via', 'was', 'we', 'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves']

    vect = TfidfVectorizer(stop_words = stopwords, ngram_range = (2,2))
    tfidf = pd.DataFrame(vect.fit_transform([article1]).toarray(), columns=vect.get_feature_names())

    top_tokens = pd.DataFrame(pd.DataFrame(tfidf.sum().sort_values(ascending=False))).reset_index().rename(columns={'index':'token',0:'freq'})

    #processing article text into sentences
    
    tokens = TextBlob(article1).sentences

    sentences = []

    for sentence in tokens:
        sentences.append(str(sentence).decode('utf-8'))
        
    #split article by sentences and put in df        
    article_df = pd.DataFrame({'sentence':sentences})

    #function for scoring each sentence for importance
    def sentence_score(string):
        score = 0.0
        string = TextBlob(string)
        for word in string.split(' '):
            word = word.lower()
            if word in list(top_tokens[top_tokens.freq > top_tokens.freq.mean()]['token']):
                score = score + top_tokens[top_tokens.token == word]['freq'].iloc[0]
            else:
                score = score + 0
        return score
    
    def sentence_score3(string):
        score = []
        sentence = string.split(' ')
        for token in list(top_tokens['token']):
            for word in token.lower().split(' '):
                word = word.lower()
                if word in sentence:
                    score.append(top_tokens[top_tokens.token == token]['freq'].iloc[0])
                else:
                    score.append(0)
        return pd.Series(score).mean()*len(sentence)

    #score each sentence
    article_df['score'] = article_df.sentence.apply(sentence_score3)

    #segment article by intro, body, and end
    position = []

    article_len = len(article_df) 
    round(article_len*.10)

    intro = [1]*int(round(article_len*.15))
    end = [3]*int(round(article_len*.33))
    body = [2]*int(round(article_len - (len(intro)+len(end))))

    position = intro + body + end

    #add position column in article df
    article_df['position'] = position

    #construct list of sentences for summary using most important sentences from each position
    try:
        summary = [article_df[article_df.position == 1].sort_values(by='score', ascending = False).sentence.iloc[0], 
                   article_df[article_df.position == 2].sort_values(by='score', ascending = False).sentence.iloc[0], 
                   article_df[article_df.position == 3].sort_values(by='score', ascending = False).sentence.iloc[0]]
        s = ' '
        summarized = s.join(summary)
        
    except:
        summarized = ' '.join(article_df.sentence)

    #join the sentences together and fix punctuation
    pattern = re.compile('\[[0-9]+\]')
    summarized = re.sub(pattern,'', summarized)
    
    if summarized[-1] != '.':
        summarized = summarized + '.'
    else:
        pass
    
    summarized = summarized.replace(' ,',', ').replace(' .','.').replace(' \'s', '\'s').replace('( ','(').replace(' )',')').replace(' "','').replace(' :',':').replace(' ;',';')

    
    
    #compression
    summ_len = len(summarized)
    long_len = len(article1)
    compression = (1 - float(summ_len)/long_len)*100
    
    
    #return article_link, summarized
    print title + '\n' + story_link + '\n\n' + summarized + '\n\n' + 'Compression Stats:\nOriginal Article Length: ' + str(long_len) + ' characters\nSummary Length: ' + str(summ_len) + ' characters\n' + str(compression) + '% compressed'
    
    
    

In [4]:
NYT(0)

Senators and Spy Chief Rebut Trump on Russia Findings
http://www.nytimes.com/2017/01/05/us/politics/taking-aim-at-trump-leaders-strongly-affirm-findings-on-russian-hacking.html

WASHINGTON — Rebuffing efforts by President-elect Donald J. Trump to cast doubt on Russian interference in the presidential election, top intelligence officials and senators from both parties on Thursday issued a forceful affirmation of the findings. Though Mr. Clapper and most Republican senators were careful to avoid antagonizing the president-elect directly, the hearing spoke to the searing rift Mr. Trump has threatened to create between the incoming administration and the intelligence officials tasked with informing it. Ms. McCaskill said there would be “howls from the Republican side of the aisle” if a Democrat had spoken about intelligence officials as Mr. Trump has.

Compression Stats:
Original Article Length: 6678 characters
Summary Length: 681 characters
89.8023360288% compressed
